#  Práctica Obligatoria - Parte IV - Visualización

***<p style="text-align:center;">Aprendizaje Automático II</p>***
***<p style="text-align:center;">Visualización</p>***

En esta parte, cargarás un modelo CNN entrenado y visualizarás los parámetros y sus activaciones.

### Evaluación - 1.5/10 puntos

Puntuación de cada parte sobre el total de la práctica:
- **[Ejercicio 1]** 1.5 puntos.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import transforms

from gts_dataset import GTS

In [2]:
!unzip /content/data.zip -d /content/

Se han truncado las últimas 5000 líneas del flujo de salida.
 extracting: /content/data/test_prepared/06646.png  
 extracting: /content/data/test_prepared/08013.png  
 extracting: /content/data/test_prepared/05006.png  
 extracting: /content/data/test_prepared/01747.png  
 extracting: /content/data/test_prepared/08006.png  
 extracting: /content/data/test_prepared/03231.png  
 extracting: /content/data/test_prepared/05240.png  
 extracting: /content/data/test_prepared/03732.png  
 extracting: /content/data/test_prepared/09868.png  
 extracting: /content/data/test_prepared/05120.png  
 extracting: /content/data/test_prepared/12523.png  
 extracting: /content/data/test_prepared/10590.png  
 extracting: /content/data/test_prepared/11227.png  
 extracting: /content/data/test_prepared/04026.png  
 extracting: /content/data/test_prepared/06894.png  
 extracting: /content/data/test_prepared/06080.png  
 extracting: /content/data/test_prepared/12147.png  
 extracting: /content/data/test_prepar

## Importa las funciones que has creado desde `utils.py`:

In [3]:
from utils import count_trainable_params, train, evaluate, train_and_evaluate, save_full_model, load_full_model, plot_loss_accuracy, plot_confusion_matrix, plot_error_per_class

## Carga el dataset GTS  y crea los dataloaders:

In [5]:
# Definir las transformaciones y conversión a tensor
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = GTS(csv_file='train.csv', root_dir='./data', transform=transform)
valid_dataset = GTS(csv_file='valid.csv', root_dir='./data', transform=transform)
test_dataset = GTS(csv_file='test.csv', root_dir='./data', transform=transform)

device = 'cpu'
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


# Carga el modelo `CNN` entrenado (`cnn.pth`):

In [6]:
from models import CNN

# Crear una instancia de la red
cnn_model = CNN(output_dim=43)

# Ver el número de parámetros entrenables
print("Número de parámetros entrenables:", count_trainable_params(cnn_model))


Número de parámetros entrenables: 26603


## **[Ejercicio 1]** Visualiza los filtros de la primera capa convolucional y sus activaciones para una imagen dada:

Para acceder a los filtros, puedes hacer uso de la notación por punto:

`model_cnn.conv1.weight.data`

Para visualizar las activaciones, deberás ejecutar cada función del bloque convolucional que necesites (conv --> bn --> relu).

In [9]:
from ipywidgets import interact, IntSlider
import torch
import matplotlib.pyplot as plt

def plot_filter_and_activation(model, image, device='cpu'):
    """
    Visualiza la imagen original, un filtro y su activación correspondiente.
    """
    model.eval()  # Asegurar modo evaluación
    image = image.unsqueeze(0).to(device)  # Asegurarse de que la imagen esté en formato de batch y en el dispositivo correcto

    with torch.no_grad():
        # Calcular activaciones de la primera capa
        activations = model.conv1(image)  # Obtener las activaciones de la primera capa convolucional

    # Procesar filtros y activaciones para visualización
    filters = model.conv1.weight.data  # Filtros de la primera capa

    image_np = image.squeeze(0).cpu().permute(1, 2, 0).numpy()  # Imagen original en formato HWC para visualizar

    # Asegurar valores entre 0 y 1 para la imagen original
    image_np = (image_np - image_np.min()) / (image_np.max() - image_np.min())

    # Unificar el número de elementos para el slider
    num_elements = min(filters.shape[0], activations.shape[1])

    def plot_image_filter_activation(index):
        """
        Visualiza la imagen original, un filtro y su activación correspondiente.
        """
        # Configurar la figura
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        # Mostrar imagen original
        axs[0].imshow(image_np)
        axs[0].set_title("Imagen Original")
        axs[0].axis('off')

        # Mostrar filtro
        filter_img = filters[index, :, :, :].cpu().permute(1, 2, 0)  # Filtro de la primera capa
        filter_img = (filter_img - filter_img.min()) / (filter_img.max() - filter_img.min())
        axs[1].imshow(filter_img)
        axs[1].set_title(f"Filtro {index+1}")
        axs[1].axis('off')

        # Mostrar activación
        activation_img = activations[0, index, :, :].cpu().numpy()  # Activación correspondiente al filtro
        axs[2].imshow(activation_img, cmap='gray')
        axs[2].set_title(f"Activación {index+1}")
        axs[2].axis('off')

        plt.tight_layout()
        plt.show()

    # Crear un slider interactivo
    interact(plot_image_filter_activation, index=IntSlider(min=0, max=num_elements-1, step=1, description="Índice"))

# Ejemplo: Usar la visualización unificada con imagen original
example_image, _ = test_dataset[0]  # Seleccionar una imagen del conjunto de prueba
plot_filter_and_activation(cnn_model, example_image)


interactive(children=(IntSlider(value=0, description='Índice', max=15), Output()), _dom_classes=('widget-inter…

In [10]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

def shift_image_and_visualize_activations(model, image, device='cpu'):
    """
    Visualiza cómo las activaciones cambian al desplazar la imagen horizontal y verticalmente.
    """
    model.eval()  # Asegurar modo evaluación
    with torch.no_grad():
        # Calcular activaciones de la primera capa para la imagen original
        x = image.unsqueeze(0).to(device)  # Añadir dimensión de batch
        activations = model.conv1(x)  # Activaciones de la primera capa convolucional

    # Obtener los filtros de la primera capa
    filters = model.conv1.weight.data  # Filtros de la primera capa

    # Procesar la imagen para la visualización
    image_np = image.permute(1, 2, 0).cpu().numpy()  # Imagen en formato HWC

    # Normalizar la imagen original para visualización
    image_np = (image_np - image_np.min()) / (image_np.max() - image_np.min())

    def shift_and_plot_activation(horizontal_shift, vertical_shift):
        """
        Traslada la imagen en las direcciones horizontal y vertical, y visualiza el cambio en las activaciones.
        """
        # Desplazar la imagen horizontal y verticalmente
        shifted_image = np.roll(image_np, horizontal_shift, axis=1)  # Desplazamiento horizontal
        shifted_image = np.roll(shifted_image, vertical_shift, axis=0)  # Desplazamiento vertical

        # Configurar la figura
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        # Mostrar imagen original
        axs[0].imshow(image_np)
        axs[0].set_title("Imagen Original")
        axs[0].axis('off')

        # Mostrar imagen desplazada
        axs[1].imshow(shifted_image)
        axs[1].set_title(f"Imagen Desplazada (Horizontal: {horizontal_shift}, Vertical: {vertical_shift})")
        axs[1].axis('off')

        # Calcular activaciones para la imagen desplazada
        shifted_image_tensor = torch.tensor(shifted_image).permute(2, 0, 1).unsqueeze(0).float()  # Convertir a tensor
        shifted_activations = F.relu(model.bn1(model.conv1(shifted_image_tensor.to(device)))).cpu().squeeze(0)

        # Mostrar activación de la imagen desplazada
        activation_img = shifted_activations[0, :, :].detach().numpy()  # Usar la primera activación
        axs[2].imshow(activation_img, cmap='inferno')
        axs[2].set_title(f"Activación Después Desplazamiento")
        axs[2].axis('off')

        plt.tight_layout()
        plt.show()

    # Crear sliders interactivos para el desplazamiento horizontal y vertical
    max_shift = 10  # Máximo desplazamiento en píxeles
    interact(shift_and_plot_activation,
             horizontal_shift=IntSlider(min=-max_shift, max=max_shift, step=1, description="Desp. Hor"),
             vertical_shift=IntSlider(min=-max_shift, max=max_shift, step=1, description="Desp. Vert"))

# Ejemplo: Usar la visualización unificada con imagen original
example_image, _ = test_dataset[0]  # Seleccionar una imagen del conjunto de prueba
shift_image_and_visualize_activations(cnn_model, example_image)


interactive(children=(IntSlider(value=0, description='Desp. Hor', max=10, min=-10), IntSlider(value=0, descrip…